Quadratic Sieve Implementation

In [5]:
import math
from itertools import product
import numpy as np
import time

Step 1: Select an integer *n* and a corresponding bound B.

In [12]:
import math

n = 16921456439215439701
# n = 338728169412940137585419303
# n = 46839566299936919234246726809

# Function to compute a small term going to 0 as n increases
def little_o_of_1(n):
    return 1 / math.log(n)

# Compute the bound B
temp = math.log(n) * math.log(math.log(n))
B = int(math.exp(float(0.5 + little_o_of_1(n)) * math.sqrt(temp)))

# Compute the smoothness step bound
smoothness_step_bound = int(math.exp(float(1.0 + little_o_of_1(n)) * math.sqrt(temp)))

# Print the bound B in a presentable format
print(f"The bound B is: {B}")

The bound B is: 871


Step 2: Get all primes up to B to form set S of primes.


In [14]:
def sieve_of_atkin(limit):
    # Return an empty list if the limit is less than 2 (no primes)
    if limit < 2:
        return []

    # Initialization of the sieve array with False values
    sieve = [False] * (limit + 1)
    sqrt_limit = int(limit ** 0.5)

    # Step 1: Mark sieve[n] as True based on specific quadratic forms and conditions
    for x in range(1, sqrt_limit + 1):
        for y in range(1, sqrt_limit + 1):
            # n = (4 * x^2) + (y^2) with n % 12 == 1 or n % 12 == 5
            n = 4 * x * x + y * y
            if n <= limit and (n % 12 == 1 or n % 12 == 5):
                sieve[n] = not sieve[n]

            # n = (3 * x^2) + (y^2) with n % 12 == 7
            n = 3 * x * x + y * y
            if n <= limit and n % 12 == 7:
                sieve[n] = not sieve[n]

            # n = (3 * x^2) - (y^2) with n % 12 == 11 and x > y
            n = 3 * x * x - y * y
            if x > y and n <= limit and n % 12 == 11:
                sieve[n] = not sieve[n]

    # Step 2: Mark all multiples of squares of primes as False
    for a in range(5, sqrt_limit + 1):
        if sieve[a]:
            for b in range(a * a, limit + 1, a * a):
                sieve[b] = False

    # Step 3: Construct the list of primes
    primes = [2]  # Include 2, the smallest prime number
    primes += [i for i in range(3, limit + 1) if sieve[i]]

    return primes

# Generate primes up to the bound B
S = sieve_of_atkin(B)

# Print the number of primes found and a sample of the primes
print(f"Number of primes found up to {B}: {len(S)}")
print(f"Sample of primes (without 3): {S[:10]}")  # Print the first 10 primes as a sample

Number of primes found up to 871: 149
Sample of primes (without 3): [2, 5, 7, 11, 13, 17, 19, 23, 29, 31]


Step 3: Find B-smooth squares.
  - As part of the process, we create the intended matrix of prime powers for each B-smooth value, used to find x^2 equiv y^2.

In [20]:
import math
import time

def gen_x(n, p, r):
    # Generate a list of candidate numbers based on the given parameters
    sqrt_n = math.sqrt((1 + 2 * r) * n)
    sqrt_2n = math.sqrt((2 + 2 * r) * n)
    return [math.ceil(sqrt_n + i) for i in range(p + 1)] + \
           [math.ceil(sqrt_2n + i) for i in range(p + 1)]

def find_b_smooths(n, S, B):
    # Compute logarithms and initialize parameters
    log_n = math.log(n)
    primes = [p for p in S if p <= B]
    log_primes = {p: math.log(p) for p in primes}
    log_threshold = log_n - 0.5

    smooth_set = set()  # Track unique B-smooth numbers
    smooth_set_list = []
    power_matrix = []
    r = 0
    target_smooth_count = 1.3 * len(primes)  # Estimate to ensure enough smooth numbers
    print(f"Target number of smooth numbers: {math.floor(target_smooth_count)}")

    # Search for B-smooth numbers until the target count is reached
    while len(smooth_set_list) < target_smooth_count:
        x_list = gen_x(n, 10, r)  # Generate potential smooth numbers
        for x in x_list:
            xsq = x**2 % n
            log_xsq = math.log(xsq) if xsq > 1 else 0
            powers = [0] * len(primes)
            temp_xsq = xsq

            if log_xsq > log_threshold:
                continue  # Skip if too large to be B-smooth

            for j, prime in enumerate(primes):
                while temp_xsq % prime == 0:
                    powers[j] += 1
                    temp_xsq //= prime
                    log_xsq -= log_primes[prime]
                    if log_xsq < -0.5:
                        break
                if log_xsq < -0.5:
                    break

            if temp_xsq == 1 and x not in smooth_set:
                power_matrix.append(powers)
                smooth_set_list.append(x)
                smooth_set.add(x)
        r += 1  # Increment to search in the next iteration

    return smooth_set_list, power_matrix

# Track the start time
start_time = time.time()

# Find B-smooth numbers and their power matrix
smooth_list, power_matrix = find_b_smooths(n, S, B)

# Track the end time
end_time = time.time()

# Calculate and print runtime
runtime = end_time - start_time
print(f"Runtime: {runtime:.2f} seconds")

# Print results: number of smooth numbers and a sample of the power matrix
print(f"Number of B-smooth numbers found: {len(smooth_list)}")
print(f"First row of power matrix: {power_matrix[0]}")

Target number of smooth numbers: 193
Runtime: 3.30 seconds
Number of B-smooth numbers found: 194
Sample of power matrix (first row): [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Step 4: Row-reduce matrix and find a linear dependence.

In [23]:
import numpy as np
import math

# Convert power_matrix to a numpy array and prepare for Gaussian elimination
A = np.array(power_matrix)
power_copy = A.copy()
A = (np.transpose(A)) % 2

def gaussian_elimination(mat):
    # Perform Gaussian elimination on a matrix in GF(2)
    num_rows, num_cols = mat.shape
    row = 0
    pivot_positions = []  # Track positions of pivots

    for col in range(num_cols):
        if row >= num_rows:
            break
        pivot_row = None
        for r in range(row, num_rows):
            if mat[r, col] == 1:
                pivot_row = r
                break
        if pivot_row is not None:
            # Swap rows
            mat[[row, pivot_row]] = mat[[pivot_row, row]]
            # Eliminate other rows
            for r in range(num_rows):
                if r != row and mat[r, col] == 1:
                    mat[r] ^= mat[row]  # Update the matrix using XOR
            pivot_positions.append((row, col))
            row += 1
    return mat[:row], pivot_positions

def find_null_space(mat, pivot_positions):
    # Compute the null space of the matrix
    num_rows, num_cols = mat.shape
    free_vars = [j for j in range(num_cols) if all(j != col for _, col in pivot_positions)]
    null_space_basis = []

    for free_var in free_vars:
        vec = np.zeros(num_cols, dtype=int)
        vec[free_var] = 1
        for i, col in pivot_positions:
            if i < num_rows:
                vec[col] = mat[i, free_var]
        null_space_basis.append(vec)
    return null_space_basis

# Perform Gaussian elimination to obtain the reduced row echelon form
result, pivot_positions = gaussian_elimination(A)
#print("Reduced row echelon form:")
#print(result)

# Find the null space basis
null_space_basis = find_null_space(result, pivot_positions)
#print("Null space basis:")
#for vec in null_space_basis:
 #   print(vec)

# Process the null space basis to find factors
for v in null_space_basis:
    power_matrix_subset = [x for x, check in zip(power_matrix, v) if check == 1]
    x_list = [x for x, check in zip(smooth_list, v) if check == 1]
    x_final = 1
    for num in x_list:
        x_final = (x_final * num) % n

    y_final = 1
    for col_index, col in enumerate(zip(*power_matrix_subset)):
        column_sum = sum(col)
        column_product = S[col_index] ** (column_sum // 2)
        y_final = (y_final * column_product) % n

    if x_final != y_final:
        print("x_final mod n:", x_final)
        print("y_final mod n:", y_final)
        print("x^2 mod n:", (x_final ** 2) % n)
        print("y^2 mod n:", (y_final ** 2) % n)
        print("Factors:")
        factor = math.gcd(x_final - y_final, n)
        print("GCD of (x_final - y_final) and n:", factor)
        if factor != 1 and factor != n:
            other_factor = n // factor
            print("Non-trivial factor found:", factor)
            print("Other factor:", other_factor)
            break
else:
    print("No non-trivial factors found.")

x_final mod n: 1794158484905772962
y_final mod n: 15127297954309666739
x^2 mod n: 3984857010854416300
y^2 mod n: 3984857010854416300
Factors:
GCD of (x_final - y_final) and n: 1
x_final mod n: 6084102607933502758
y_final mod n: 10837353831281936943
x^2 mod n: 3670553858024071418
y^2 mod n: 3670553858024071418
Factors:
GCD of (x_final - y_final) and n: 1
x_final mod n: 12333850556098009966
y_final mod n: 8692051646593151452
x^2 mod n: 12382236686548130155
y^2 mod n: 12382236686548130155
Factors:
GCD of (x_final - y_final) and n: 2860486313
Non-trivial factor found: 2860486313
Other factor: 5915587277
